In [0]:
from pyspark.sql import functions as F

try:
    # 1. Carregar tabelas da camada Silver
    df_pedidos_total = spark.table("medalhao.silver.pedido_total")
    df_consumidores = spark.table("medalhao.silver.ft_consumidores")

    # 2. Juntar (join) as fontes de dados
    # (ft_pedido_total já tem id_consumidor, então o join é simples)
    df_joined = df_pedidos_total.join(
        df_consumidores,
        on="id_consumidor",
        how="left"
    )

    # 3. Selecionar e renomear colunas para a tabela fato
    # (Conforme especificado na tabela do PDF)
    df_gold = df_joined.select(
        F.col("id_pedido"),
        F.col("id_consumidor"),
        F.col("valor_total_pago_brl").alias("valor_total_pedido_brl"),
        F.col("cidade"),
        F.col("estado"),
        F.col("data_pedido")
    )

    # 4. Salvar a nova tabela fato na camada Gold
    df_gold.write.mode("overwrite").saveAsTable("medalhao.gold.ft_vendas_consumidor_local")
    
    print("Tabela medalhao.gold.ft_vendas_consumidor_local criada.")

except Exception as e:
    print(f"falha ao processar gold.ft_vendas_consumidor_local: {e}")

In [0]:
# Query SQL para criar a view agregada
query_view = """
    CREATE OR REPLACE VIEW medalhao.gold.view_total_compras_por_consumidor AS
    SELECT 
        cidade,
        estado,
        COUNT(id_pedido) AS quantidade_vendas,
        SUM(valor_total_pedido_brl) AS valor_total_localidade
    FROM medalhao.gold.ft_vendas_consumidor_local
    GROUP BY cidade, estado
"""

try:
    spark.sql(query_view)
    print("view medalhao.gold.view_total_compras_por_consumidor criada.")
except Exception as e:
    print(f"erro em 1.2: {e}")

In [0]:
# Validando 1.2: Total de vendas por estado

spark.sql("""
    SELECT 
        estado,
        SUM(valor_total_localidade) AS total_vendas
    FROM medalhao.gold.view_total_compras_por_consumidor
    GROUP BY estado
    ORDER BY total_vendas DESC
    LIMIT 5
""").show()